In [1]:
import os
import json
import pandas as pd

## Dataset

#### json 파일 영한 텍스트 추출

In [ ]:
def extract_data_from_json(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)

    extracted_data = []

    # 데이터를 extracted_data 리스트에 추가하는 과정을 포함
    for item in data['data']:
        en = item.get('en', '')
        ko = item.get('ko', '')
        extracted_data.append({'en': en, 'ko': ko})  # 수정된 부분

    df = pd.DataFrame(extracted_data)

    return df

In [ ]:
file_path = '/content/drive/MyDrive/kpmg/en_ko_data/'

train_path = file_path + '기술과학분야_한영_train_set.json'
test_path = file_path + '기술과학분야_한영_valid_set.json'

train_df = extract_data_from_json(train_path)
test_df = extract_data_from_json(test_path)

In [ ]:
train_df.head()

,en,ko
0,A medical group has even filed a federal court...,한 의료단체는 연방법원에 보건부 지침 취소를 요구하는 소송을 제기하기도 했다.
1,It should be noted that there are significant ...,특정 지역에서는 순위 변동이 크게 나타나고 있는 점에 주목해야 한다.
2,"Scholars from Japan, China, and Korea will loo...",일본·중국·한국의 학자들이 참석해 개발학의 현황부터 유엔의 '2030 지속가능 개발...
3,The comparators 1235 and 1237 may be expressed...,"비교기(1235 및 1237)는 설정에 따라 Relu 활성함수로 나타낼 수 있으며,..."
4,The program is credited with focusing urban re...,이 프로그램은 도시 재생의 초점을 낙후 지역이나 불량 주택을 철거하기보다는 근린 지...


In [ ]:
test_df.head()

,en,ko
0,The first point and the second point may inclu...,상기 제1 지점 및 상기 제2 지점은 표시 장치(160) 내에 위치하는 서로 상이한...
1,Exploratory learning will also have a definiti...,탐색 학습 또한 기업의 경영 성과에 정의 영향을 미칠 것이다.
2,High tariffs in the range of 20% to 40% are ap...,자국과 자유 무역 협정이 체결되지 않은 국가들로부터 수입된 완성차와 자국에 생산 시...
3,"If this is acknowledged, there will arise a pr...",만일 이를 인정한다면 소비자의 의사에 기하지 않은 의무(반송 의무)가 사업자의 일방...
4,The proxy solicitation process is very similar...,위임장 권유 절차는 주주 총회의 소집 절차와 매우 유사한 방식이나 소집 절차와 동시...


In [ ]:
# 필요에 따라 TSV 형태로 저장하려면 sep='\t' 옵션을 사용
train_df.to_csv('tech_train_dataset.tsv', sep='\t', index=False)
test_df.to_csv('tech_test_dataset.tsv', sep='\t', index=False)

허깅페이스 데이터셋 생성

In [ ]:
from datasets import Dataset

In [ ]:
dataset_train = Dataset.from_pandas(train_df)
dataset_test = Dataset.from_pandas(test_df)

In [ ]:
dataset_train, dataset_test

(Dataset({
     features: ['en', 'ko', '__index_level_0__'],
     num_rows: 960115
 }),
 Dataset({
     features: ['en', 'ko'],
     num_rows: 150018
 }))

In [ ]:
from sklearn.model_selection import train_test_split

train_df, valid_df = train_test_split(train_df, test_size=0.2, random_state=42)

# 분할된 데이터셋의 크기 확인
print(f"Train set size: {len(train_df)}")
print(f"Validation set size: {len(valid_df)}")
print(f"Test set size: {len(test_df)}")

Train set size: 960115
Validation set size: 240029
Test set size: 150018


In [ ]:
# TSV 파일로 저장
train_df.to_csv('train.tsv', sep='\t', index=False)
valid_df.to_csv('valid.tsv', sep='\t', index=False)
test_df.to_csv('test.tsv', sep='\t', index=False)

In [ ]:
from datasets import load_dataset

dataset = load_dataset('csv', data_files={'train': 'train.tsv', 'validation': 'valid.tsv', 'test': 'test.tsv'}, delimiter='\t')

print(dataset)

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['en', 'ko'],
        num_rows: 960115
    })
    validation: Dataset({
        features: ['en', 'ko'],
        num_rows: 240029
    })
    test: Dataset({
        features: ['en', 'ko'],
        num_rows: 150018
    })
})


In [ ]:
# train 스플릿에서 영어 3개와 한국어 3개 샘플을 가져온다.
print(dataset['train']['en'][:3], dataset['train']['ko'][:3])

['Chapter 2 discusses major hypotheses and previous studies, and Chapter 3 presents the results of model setting, data collection, and empirical analysis.', 'In addition to military forces, the government will send experts from environmental agencies such as the Institute for Environment and Renewable Natural Resources (Ibama) and the Shiku Menzies Institute for Biodiversity (ICMBio) to the Pantanau area to fight fires and protect ecosystems.', 'Since only one entity establishes the key from which all other keys originate, no other entity may check and verify that all requirements have been met.'] ['제2장은 주요 가설과 선행 연구를 논의하고, 제3장에서는 모형 설정, 자료 수집 및 실증 분석 결과를 제시한다.', '정부는 군 병력 외에 환경·재생 가능 천연자원 연구소(Ibama)와 시쿠 멘지스 생물종 다양성 연구소(ICMBio) 등 환경 관련 기관의 전문가들을 판타나우 지역에 보내 화재 진압과 생태계 보호에 나설 것으로 알려졌다.', '모든 다른 키가 비롯되는 키를 오직 하나의 엔티티가 설정하기 때문에, 모든 요구가 충족되었는지를 다른 어떤 엔티티가 체크 및 확인할 수 없다.']


## Data 불러오기

In [4]:
train_df = pd.read_csv('/content/drive/MyDrive/kpmg/en_ko_data/train.tsv', sep='\t')
valid_df = pd.read_csv('/content/drive/MyDrive/kpmg/en_ko_data/valid.tsv', sep='\t')
test_df = pd.read_csv('/content/drive/MyDrive/kpmg/en_ko_data/test.tsv', sep='\t')

In [5]:
from datasets import load_dataset

path = '/content/drive/MyDrive/kpmg/en_ko_data/'
# 로컬 파일 시스템에서 데이터셋 로드
dataset = load_dataset('csv', data_files={'train': path+'train.tsv', 'validation': path+'valid.tsv', 'test': path+'test.tsv'}, delimiter='\t')

print(dataset)

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['en', 'ko'],
        num_rows: 682756
    })
    validation: Dataset({
        features: ['en', 'ko'],
        num_rows: 170690
    })
    test: Dataset({
        features: ['en', 'ko'],
        num_rows: 133336
    })
})


In [6]:
# 테스트 세트를 10000 문장으로 축소
reduced_test_dataset = dataset['test'].shuffle(seed=42).select(range(10000))

# 데이터셋 업데이트
dataset['test'] = reduced_test_dataset

# 결과 출력
print("Updated DatasetDict:")
print(dataset)

Updated DatasetDict:
DatasetDict({
    train: Dataset({
        features: ['en', 'ko'],
        num_rows: 682756
    })
    validation: Dataset({
        features: ['en', 'ko'],
        num_rows: 170690
    })
    test: Dataset({
        features: ['en', 'ko'],
        num_rows: 10000
    })
})


## Huggingface Tokenizer

In [7]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import DataCollatorForSeq2Seq
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
from datasets import load_dataset, load_metric

import numpy as np
import torch
import multiprocessing

In [8]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [9]:
model_ckpt = "KETI-AIR/ke-t5-base"
max_token_length = 64

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

In [11]:
dataset['train'][10]['en'], dataset['train'][10]['ko']

('The more information and data you assign, the more precise climate predictions you can make.',
 '더 많은 정보와 자료를 대입할 수록 더 정밀한 기후예측이 가능하기 때문이다.')

In [12]:
tokenized_sample_en = tokenizer(dataset['train'][10]['en'],
                                max_length=max_token_length,
                                padding=True, truncation=True)
tokenized_sample_en

{'input_ids': [55, 139, 1077, 13, 1161, 106, 64, 22693, 4, 5, 139, 44963, 5571, 47425, 106, 147, 464, 3, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [13]:
tokenized_sample_ko = tokenizer(dataset['train'][10]['ko'],
                                max_length=max_token_length,
                                padding=True, truncation=True)
tokenized_sample_ko

{'input_ids': [121, 253, 25511, 3970, 16111, 88, 18037, 121, 8386, 50, 16152, 50991, 15, 48540, 501, 3, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [14]:
tokenizer(dataset['train'][:3]['en'],
          max_length=max_token_length,
          padding=True, truncation=True)

{'input_ids': [[15108, 1689, 2383, 13, 1161, 3958, 7904, 615, 67, 65, 5, 2107, 14, 16, 1161, 1000, 4, 6417, 1245, 16, 1838, 1949, 3094, 1161, 1689, 2383, 3, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [215, 5, 40966, 14, 110, 4, 5, 62450, 14402, 13, 42671, 58181, 5924, 1028, 1968, 5, 275, 1310, 14, 32981, 4, 8190, 5, 1090, 14, 2251, 54, 992, 240, 1221, 240, 47695, 10, 543, 2251, 3, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [49591, 1773, 1387, 41446, 103, 806, 16, 2924, 14, 5, 61372, 726, 5, 43906, 482, 41387, 4, 5, 61372, 1838, 48295, 726, 5, 43906, 482, 41387, 43, 65, 16, 11401, 14, 14894, 4329, 10, 29908, 5011, 3, 39926, 1835, 15441, 4, 267, 62, 16, 12758, 5400, 89, 2264, 18, 61372, 20, 5, 5181, 726, 174, 1838, 48295, 8, 3, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [15]:
pd.DataFrame(
    [
        tokenized_sample_ko['input_ids'],
        tokenizer.convert_ids_to_tokens(tokenized_sample_ko['input_ids'])
    ], index=('ids', 'tokens')
)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
ids,121,253,25511,3970,16111,88,18037,121,8386,50,16152,50991,15,48540,501,3,1
tokens,▁더,▁많은,▁정보와,▁자료를,▁대입,할,▁수록,▁더,▁정밀,한,▁기후,예측,이,▁가능하기,▁때문이다,.,</s>


In [16]:
pd.DataFrame(
    [
        tokenized_sample_en['input_ids'],
        tokenizer.convert_ids_to_tokens(tokenized_sample_en['input_ids'])
    ], index=('ids', 'tokens')
)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
ids,55,139,1077,13,1161,106,64,22693,4,5,139,44963,5571,47425,106,147,464,3,1
tokens,▁The,▁more,▁information,▁and,▁data,▁you,▁as,sign,",",▁the,▁more,▁precise,▁climate,▁predictions,▁you,▁can,▁make,.,</s>


In [17]:
def convert_examples_to_features(examples):

    model_inputs = tokenizer(examples['ko'],
                             text_target=examples['en'],
                             max_length=max_token_length, truncation=True)

    return model_inputs

In [18]:
NUM_CPU = multiprocessing.cpu_count()
NUM_CPU

8

In [ ]:
tokenized_datasets = dataset.map(convert_examples_to_features,
                                 batched=True,
                                 remove_columns=dataset["train"].column_names,
                                 num_proc=NUM_CPU)

In [20]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 682756
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 170690
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 10000
    })
})

In [22]:
print( '원 데이터    :', dataset['train'][10]['ko'] )
print( '처리 후 데이터:', tokenized_datasets['train'][10]['input_ids'] )
print( '토큰화       :', tokenizer.convert_ids_to_tokens(tokenized_datasets['train'][10]['input_ids']) )

print('\n')
print( '원 데이터    :', dataset['train'][10]['en'] )
print( '처리 후 데이터:', tokenizer.convert_ids_to_tokens(tokenized_datasets['train'][10]['labels']) )
print( '토큰화       :', tokenized_datasets['train'][10]['labels'] )

원 데이터    : 더 많은 정보와 자료를 대입할 수록 더 정밀한 기후예측이 가능하기 때문이다.
처리 후 데이터: [121, 253, 25511, 3970, 16111, 88, 18037, 121, 8386, 50, 16152, 50991, 15, 48540, 501, 3, 1]
토큰화       : ['▁더', '▁많은', '▁정보와', '▁자료를', '▁대입', '할', '▁수록', '▁더', '▁정밀', '한', '▁기후', '예측', '이', '▁가능하기', '▁때문이다', '.', '</s>']


원 데이터    : The more information and data you assign, the more precise climate predictions you can make.
처리 후 데이터: ['▁The', '▁more', '▁information', '▁and', '▁data', '▁you', '▁as', 'sign', ',', '▁the', '▁more', '▁precise', '▁climate', '▁predictions', '▁you', '▁can', '▁make', '.', '</s>']
토큰화       : [55, 139, 1077, 13, 1161, 106, 64, 22693, 4, 5, 139, 44963, 5571, 47425, 106, 147, 464, 3, 1]


## Model Training

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

In [24]:
encoder_inputs = tokenizer(
    ["상기 제1 지점 및 상기 제2 지점은 표시 장치(160) 내에 위치하는 서로 상이한 지점들을 포함할 수 있다."],
    return_tensors="pt"
)['input_ids'].to(device)

decoder_targets = tokenizer(
    ["The first point and the second point may include different points located in the display device 160."],
    return_tensors="pt"
)['input_ids'].to(device)

print( encoder_inputs )
print( decoder_targets )

decoder_inputs = model._shift_right(decoder_targets)

pd.DataFrame(
    [
        tokenizer.convert_ids_to_tokens(decoder_targets[0]),
        tokenizer.convert_ids_to_tokens(decoder_inputs[0])
    ],
    index=('decoder target', 'decoder input')
)

tensor([[22722,   190,   240,  8240,    78, 22722,   190,   148,  8240,    19,
          4132, 10272,    26, 29561,    18,  4254,  3449,    74,  1423, 55084,
          8240,   383,  2684,    88,    35,    32,     3,     1]],
       device='cuda:0')
tensor([[   55,   275,  1448,    13,     5,   870,  1448,   615,  1897,  1272,
          1541,  6716,    20,     5,  7028,  5870, 11429,     3,     1]],
       device='cuda:0')


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
decoder target,▁The,▁first,▁point,▁and,▁the,▁second,▁point,▁may,▁include,▁different,▁points,▁located,▁in,▁the,▁display,▁device,▁160,.,</s>
decoder input,<pad>,▁The,▁first,▁point,▁and,▁the,▁second,▁point,▁may,▁include,▁different,▁points,▁located,▁in,▁the,▁display,▁device,▁160,.


In [26]:
# forward pass
outputs = model(input_ids=encoder_inputs,
                decoder_input_ids=decoder_inputs,
                labels=decoder_targets)

outputs['encoder_last_hidden_state'].shape

torch.Size([1, 28, 768])

In [27]:
outputs['logits'].shape

torch.Size([1, 19, 64128])

In [ ]:
tokenizer.convert_ids_to_tokens( torch.argmax(outputs['logits'][0], axis=1).cpu().numpy() )

## Collator

In [29]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
# 콜레이터로 padding & shift

batch = data_collator(
    [tokenized_datasets["train"][i] for i in range(1, 3)]
)

batch

## Metric

In [ ]:
import evaluate

metric = evaluate.load("sacrebleu")

In [32]:
def compute_metrics(eval_preds):
    preds, labels = eval_preds

    if isinstance(preds, tuple):
        preds = preds[0]

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    return result

# Model Train

In [ ]:
output_dir="/content/drive/MyDrive/kpmg/en_ko_data/ModelCheckpoint"

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir=output_dir,

    learning_rate=0.0005,
    weight_decay=0.01,
    per_device_train_batch_size=12,
    per_device_eval_batch_size=128,
    num_train_epochs=1,
    save_steps=1000,
    save_total_limit=2,
    evaluation_strategy="steps",
    eval_steps = 500,
    logging_strategy="steps",
    logging_steps=500,  # 로깅을 수행할 스텝 간격 설정

    predict_with_generate=True,
    fp16=False,
    gradient_accumulation_steps=2,
    report_to="none", # Wandb 로그 끄기

    load_best_model_at_end=True,  # 훈련 종료 시 최상의 모델 로드
    metric_for_best_model="loss"  # 최상의 모델을 결정하기 위한 메트릭

)

In [ ]:
trainer = Seq2SeqTrainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

Step,Training Loss,Validation Loss,Bleu
500,7.094900,3.661932,3.195983
1000,3.709200,2.683457,9.052477
1500,3.047900,2.304677,10.996290
2000,2.705100,2.073633,12.181403
2500,2.468400,1.895607,13.238561
3000,2.284300,1.780563,13.754313
3500,2.168900,1.697934,14.378674
4000,2.069600,1.622311,14.710047
4500,1.983500,1.552918,15.017341
5000,1.895600,1.491608,16.112264


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control

KeyboardInterrupt: 

In [ ]:
trainer.train()

# Test

In [ ]:
# output_dir="/content/drive/MyDrive/kpmg"
# trainer.save_model(output_dir)

In [69]:
model_dir = "/content/drive/MyDrive/kpmg/pretrained_model"
tokenizer = AutoTokenizer.from_pretrained(model_dir)
model = AutoModelForSeq2SeqLM.from_pretrained(model_dir)

In [ ]:
# Repository 생성 & model upload
REPO_NAME = 'T5-Patent-Translation-ko-en'
AUTH_TOKEN = 'my_token' # <https://huggingface.co/settings/token>

## Upload to Huggingface Hub
model.push_to_hub(
    REPO_NAME,
    use_temp_dir=True,
    token=AUTH_TOKEN
)
tokenizer.push_to_hub(
    REPO_NAME,
    use_temp_dir=True,
    token=AUTH_TOKEN
)

In [ ]:
model.cpu()

In [35]:
input_text = '본 발명은 차량의 주행환경 또는 노면조건 등에 따라 차량의 좌우 바퀴폭을 조절할 수 있는 좌우 바퀴폭 조절장치에 관한 것이다.'

In [ ]:
inputs = tokenizer(input_text, return_tensors="pt",
                   padding=True, max_length=max_token_length)

In [37]:
english = model.generate(
    **inputs,
    max_length=max_token_length,
    num_beams=5,
)

english.shape

torch.Size([1, 47])

In [38]:
[
    tokenizer.convert_tokens_to_string(
    tokenizer.convert_ids_to_tokens(english)) for english in english
]

['<pad> The present invention relates to a left and right wheel width adjustment device capable of adjusting the left and right wheel width of the vehicle according to the driving environment or road surface conditions of the vehicle.</s>']

In [39]:
from torch.utils.data import DataLoader

test_dataloader = DataLoader(
    tokenized_datasets["test"], batch_size=32, collate_fn=data_collator
)

In [40]:
test_dataloader_iter = iter(test_dataloader)
test_batch = next(test_dataloader_iter)
test_batch.keys()
test_input = { key: test_batch[key] for key in ('input_ids', 'attention_mask') }

In [41]:
koreans = model.generate(
    **test_input,
    max_length=max_token_length,
    num_beams=3,
)

In [42]:
labels =  np.where(test_batch.labels != -100, test_batch.labels, tokenizer.pad_token_id)
eng_sents = tokenizer.batch_decode(test_batch.input_ids, skip_special_tokens=True)[10:20]
references = tokenizer.batch_decode(labels, skip_special_tokens=True)[10:20]
preds = tokenizer.batch_decode( koreans, skip_special_tokens=True )[10:20]

for s in zip(eng_sents, references, preds):
    print('English   :', s[0])
    print('Reference :', s[1])
    print('Translated:', s[2])
    print('\n')

English   : 분석 서버(110)는 중고 차량 매매를 진행할 수 있는 웹 페이지의 주소를 고객에게 제공할 수 있다.
Reference : The analysis server 110 may provide the customer with the address of a web page through which a second-hand vehicle may be sold.
Translated: The analysis server 110 may provide a customer with a address of a web page that may proceed with the sale of used vehicles.


English   : 기후 또는 지형 요인과 같은 다른 요인의 결과로 잔차의 차이가 지역 수준에 존재하면 잔차는 공간 자기상관을 나타낸다.
Reference : If differences in residuals exist at the regional level as a result of other factors, such as climatic or geographic factors, then the residuals represent spatial autocorrelation.
Translated: As a result of other factors such as climate or topographical factors, if the difference in residuals exists at the regional level, the residual represents spatial self-reliance.


English   : 코로나19 확산 방지 업무로 밤낮없이 일하는 이들이 인근에 문을 연 식당이 없어 끼니를 거르면서 일하는 일이 잦다는 얘기를 듣고 안타까운 마음에 나선 일이었다.
Reference : It was sad to hear that people who work day and night to prevent the sprea

In [ ]:
labels

array([[22722,   190,   240, ...,     0,     0,     0],
       [20770,  3069,   360, ...,     0,     0,     0],
       [ 8273,    31,  2869, ...,     0,     0,     0],
       ...,
       [10272,  6043,    18, ...,     0,     0,     0],
       [36069,    75,  2235, ...,     0,     0,     0],
       [11285, 26447,    18, ...,     0,     0,     0]])

In [ ]:
preds

['18일 필리핀 주재 한국대사관에 따르면 필리핀 경찰청에서 파견돼 한국대사관 민원 사무소에서 일하는 필리핀 경찰관이 전날 코로나19 확진 판정을 받았다.',
 '에지 트림(608b)은 또한 에지(612)를 통해 커버(604b)에 빛이 진입하거나 커버(604b)를 떠나지 않도록 방지할 수 있다.',
 '이날 오전 8시께 성당에서 화재 신고가 나오자 100명의 소방관들이 현장에 있었다.',
 '이 글은 북핵 문제의 노력과 좌절에 대한 고찰에서 실질적인 결론을 도출하고자 한다.',
 '이를 위해, 신원 인증 시스템(100)은 사용자의 성문 정보를 미리 저장할 수 있고, 성문 분석 모듈을 포함할 수 있다.',
 '헬로세 발발 이후 북부 중위도 지역의 여름철 태양열 발생이 꾸준히 감소하였음을 고려하면 헬로세 몬순의 이상적인 위상은 하위도 이후에 도착할 가능성이 높다.',
 '공공과 민간의 R&D는 전반적으로 높은 중요도를 나타내며, 이 모두 조화될 때 시너지 효과가 나타났다.',
 '데이터 클록 회로(304)는 데이터 신호(310)를 주변 클록(312)에 비교하고/하거나 데이터 신호(310)의 속성을 주변 클록(312)에 비교할 수 있다.',
 '강사의 국적은 몽골, 인도네시아, 카자흐스탄, 중국 등이며, 현재 경북에 위치한 국립대학에 대학원생으로 재학 중이다.',
 '이후 국제 협상에서 조직된 지방 자치 단체의 영향력이 확대되면서 2015년 파리 협약에서도 시민사회, 민간, 금융 기관, 도시, 지방 자치 단체 등 국가 이외의 모든 이해관계자의 노력을 요구하고 있다.']

In [57]:
import evaluate

metric = evaluate.load("sacrebleu")

result = metric.compute(predictions=preds, references=references)
bleu_score = result["score"]

print("BLEU score:", bleu_score)

BLEU score: 45.641426164750754


#### 번역 예시

In [53]:
def translate_text(text):
    # 입력 텍스트를 모델이 이해할 수 있는 형태로 인코딩
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)

    # 모델을 사용하여 번역 생성
    output_sequences = model.generate(input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"],
                                      max_length=512, num_beams=5, early_stopping=True)

    # 생성된 번역 텍스트 디코딩
    translated_text = tokenizer.decode(output_sequences[0], skip_special_tokens=True)
    return translated_text

# 텍스트 입력받아 번역하기
while True:
    input_text = input()
    if input_text == '0':
        break

    translated_text = translate_text(input_text)

    print("Original Text:", input_text)
    print("Translated Text:", translated_text)

본 발명은 차량의 주행환경 또는 노면조건 등에 따라 차량의 좌우 바퀴폭을 조절할 수 있는 좌우 바퀴폭 조절장치에 관한 것이다. 내측으로 또는 외측으로 슬라이딩 이동가능하도록 상기 제1 축에 축결합되는 제2 축;을 포함하는 것을 특징으로 하는 차량의 좌우 바퀴폭 조절장치에 관한 것이다.
Original Text: 본 발명은 차량의 주행환경 또는 노면조건 등에 따라 차량의 좌우 바퀴폭을 조절할 수 있는 좌우 바퀴폭 조절장치에 관한 것이다. 내측으로 또는 외측으로 슬라이딩 이동가능하도록 상기 제1 축에 축결합되는 제2 축;을 포함하는 것을 특징으로 하는 차량의 좌우 바퀴폭 조절장치에 관한 것이다.
Translated Text: The present invention relates to a left and right wheel width adjustment device capable of adjusting the right and right wheel width of the vehicle according to the driving environment or road surface conditions of the vehicle.
0


#### 긴 번역 예시 (14초)

In [60]:
### 요약 1개에 14초 소요됨.

def sliding_window_translation(text, model, tokenizer, window_size=512, overlap=5):
    tokens = tokenizer.encode(text, return_tensors="pt")
    token_len = tokens.size(1)
    start = 0
    translated_text = ""

    while start < token_len:
        end = start + window_size
        input_ids = tokens[:, start:end]
        outputs = model.generate(input_ids, max_length=window_size, num_beams=5, early_stopping=True)
        decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)

        # 겹치는 부분 처리
        overlap_start = max(0, end-overlap)
        overlap_tokens = tokens[:, overlap_start:end]
        overlap_decoded = tokenizer.decode(overlap_tokens[0], skip_special_tokens=True)

        if start > 0:
            # 겹치는 부분에서 겹치지 않는 부분만 추가
            overlap_len = len(overlap_decoded)
            decoded_overlap_start = decoded.find(overlap_decoded)
            non_overlap_start = decoded_overlap_start + overlap_len
            translated_text += decoded[non_overlap_start:]
        else:
            translated_text += decoded

        start = end - overlap

    return translated_text

# 긴 문장 예시
long_text = """
본 발명은 날개가 없는 선풍기에 관한 것으로, 보다 상세하게는 날개없이 공기 흐름을 발생시키는 것이다.
또 이중 구조의 송풍체로 이루어져 공기 흐름 분포를 균일하게 하는 동시에, 공기 흐름의 공급 범위를 증대시킬 수 있는 날개없는 선풍기에 관한 것이다.
본 발명에 따른 날개없는 선풍기는, 일측에 외부 공기의 유입을 위한 다수의 공기유입구가 형성된다.
내부에 팬과 모터가 장착되는 본체와; 상기 본체의 상부에 설치되며 내측에 형성된 제1 송풍구를 통해 상기 본체로부터의 공기를 흡입하여 토출시키는 제1 송풍체와;
상기 제1 송풍체의 내주면 일측에 설치된 공급덕트의 상단에 연결되어 상기 제1 송풍체와 동심원상에 배치되며
내측에 제2 송풍구가 형성되는 제2 송풍체와; 상기 본체의 내측 하부에 설치되어 상기 모터의 작동을 제어하는 제어부를 포함하는 것을 특징으로 한다.
"""

# 번역 실행
translated_text = sliding_window_translation(long_text, model, tokenizer, window_size=128, overlap=10)
print("Translated Text:", translated_text)

Translated Text: The present invention relates to a tube-less airflower in which a fan and motors are mounted on the main body, and a first blower, which is installed on the upper surface of the main body, is installed to conduct and discharge air from the main body through a first blower formed on the side surface of the main body, and a 날개-less airflower, and a 날개-less airflower, and a 날개-less airflow tubes are installed in the main body, and the supply of air flow of the present in the present in the present in the air flower,The first blower and the second blower, which is connected to the upper end of the supply station installed on one side of the first blower and is disposed on the east side of the first blower and is installed on the lower side of the main body to control the operation of the motor.


In [ ]:
# 1차 훈련 결과 (리소스 문제로 중단되어 39)
# result = metric.compute(predictions=preds, references=references)
# bleu_score = result["score"]

# print("BLEU score:", bleu_score)

BLEU score: 39.440765431875874


In [64]:
import os
import sys
import json
import urllib.request


client_id = "id" # 개발자센터에서 발급받은 Client ID 값
client_secret = "secret" # 개발자센터에서 발급받은 Client Secret 값

encText = urllib.parse.quote(long_text)
data = "source=ko&target=en&text=" + encText
url = "https://openapi.naver.com/v1/papago/n2mt"

request = urllib.request.Request(url)
request.add_header("X-Naver-Client-Id",client_id)
request.add_header("X-Naver-Client-Secret",client_secret)

response = urllib.request.urlopen(request, data=data.encode("utf-8"))
rescode = response.getcode()

if(rescode==200):
    response_body = response.read()
    response_body = response_body.decode('utf-8')
    # print(response_body)

    parsed = json.loads(response_body)
    translated_text_api = parsed['message']['result']['translatedText']
    print("한영번역문:", translated_text_api)

else:
    print("Error Code:" + rescode)

한영번역문: The present invention relates to a wingless fan, and more particularly, to generate an air flow without a wing. 
It also relates to a wingless fan that consists of a dual-structure blower to equalize the distribution of air flow and increase the supply range of air flow. 
The wingless fan according to the present invention has a plurality of air inlets formed on one side for introducing external air. 
A main body equipped with a fan and motor inside; a first blower installed on the upper part of the main body and suctioning and discharging air from the main body through a first air outlet formed inside; 
It is connected to the upper end of the supply duct installed on one side of the inner circumferential surface of the first blower and placed concentrically with the first blower 
It is characterized in that it includes a second blower with a second air outlet formed inside; and a control unit installed in the lower inner part of the main body to control the operation of the mot

In [ ]:
from datasets import load_metric

metric = load_metric("bleu")
predictions = [translated_text.split('.')]
references = [[translated_text_api.split('.')]]

result = metric.compute(predictions=translated_text, references=translated_text_api)
bleu_score = result["score"]

print("BLEU score:", bleu_score)